In [ ]:
import pandas as pd
import numpy as np
import json
import requests
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from geopy.distance import geodesic
from shapely.geometry import Point, Polygon
from sklearn.preprocessing import StandardScaler
import os

# 1. Processing Crash Data and Training the AI Model

def load_crash_data(filename):
    data = pd.read_csv(filename, low_memory=False)
    data = data[['latitude', 'longitude', 'crash_sev_id', 'Crash timestamp (US/Central)']].dropna()
    return data


def identify_crash_hotspots(data, grid_size=0.01):
    data['lat_bin'] = (data['latitude'] // grid_size) * grid_size
    data['lng_bin'] = (data['longitude'] // grid_size) * grid_size

    hotspot_severity = data.groupby(['lat_bin', 'lng_bin'])['crash_sev_id'].mean().reset_index()
    return hotspot_severity


def train_model(hotspot_data):
    X = hotspot_data[['lat_bin', 'lng_bin']]
    y = hotspot_data['crash_sev_id']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error on test data: {mse}')

    return model


# 2. Fetching Routes and Evaluating with AI Model

def get_google_routes(api_key):
    base_url = "https://maps.googleapis.com/maps/api/directions/json"
    params = {
        "origin": "Austin, TX",
        "destination": "Houston, TX",
        "mode": "driving",
        "alternatives": "true",
        "key": api_key
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        routes = response.json().get("routes", [])
        print(f"Found {len(routes)} route options.")
        return routes
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return []


def calculate_safety_score(route, model):
    scores = []
    total_duration = 0

    for leg in route['legs']:
        total_duration += leg['duration']['value'] / 60  # Convert to minutes
        for step in leg['steps']:
            lat = step['end_location']['lat']
            lng = step['end_location']['lng']
            safety_score = model.predict(np.array([[lat, lng]]))[0]
            scores.append(safety_score)

    avg_score = np.mean(scores) if scores else 0
    return min(max(10 - avg_score, 1), 10), total_duration


# Putting it all together

def main():
    api_key = input("Enter your Google Maps API key: ")
    crash_data_file = 'data.csv'

    data = load_crash_data(crash_data_file)
    hotspot_data = identify_crash_hotspots(data)
    model = train_model(hotspot_data)

    routes = get_google_routes(api_key)
    
    safety_scores_and_times = [calculate_safety_score(route, model) for route in routes]
    
    for i, (route, (score, duration)) in enumerate(zip(routes, safety_scores_and_times)):
        print(f"Route {i + 1}: Safety Score {score:.2f}/10, Estimated Time: {duration:.2f} mins")
        print("Route Summary:")
        for leg in route['legs']:
            for step in leg['steps'][:3]:  # Show first few steps for brevity
                print(f"  - {step['html_instructions']} ({step['distance']['text']}, {step['duration']['text']})")
            print("...")

    safest_route_idx = np.argmax([score for score, _ in safety_scores_and_times])
    print(f"Safest Route: {safest_route_idx + 1} with Safety Score: {safety_scores_and_times[safest_route_idx][0]:.2f}/10, Estimated Time: {safety_scores_and_times[safest_route_idx][1]:.2f} mins")


if __name__ == "__main__":
    main()



Mean Squared Error on test data: 0.32457581361893867
Found 3 route options.
Route 1: Safety Score 6.68/10
Route Summary:
  - Head <b>east</b> on <b>E 5th St.</b> toward <b>Brazos St</b> (0.5 mi, 3 mins)
  - Turn <b>right</b> onto <b>S I-35 Frontage Rd</b> (177 ft, 1 min)
  - Slight <b>left</b> toward <b>N Interstate 35 Frontage Rd</b> (381 ft, 1 min)
...
Route 2: Safety Score 6.52/10
Route Summary:
  - Head <b>east</b> on <b>E 5th St.</b> toward <b>Brazos St</b> (0.5 mi, 3 mins)
  - Turn <b>right</b> onto <b>S I-35 Frontage Rd</b> (177 ft, 1 min)
  - Slight <b>left</b> toward <b>N Interstate 35 Frontage Rd</b> (381 ft, 1 min)
...
Route 3: Safety Score 6.86/10
Route Summary:
  - Head <b>east</b> on <b>E 5th St.</b> toward <b>Brazos St</b> (410 ft, 1 min)
  - Turn <b>right</b> onto <b>Brazos St</b> (358 ft, 1 min)
  - Turn <b>right</b> onto <b>E 4th St</b> (436 ft, 1 min)
...
Safest Route: 3 with Safety Score: 6.86/10


/Users/khushl/Documents/GitHub/Smart-Drive/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Users/khushl/Documents/GitHub/Smart-Drive/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Users/khushl/Documents/GitHub/Smart-Drive/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Users/khushl/Documents/GitHub/Smart-Drive/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Users/khushl/Documents/GitHub/Smart-Drive/venv/lib/python3.11/s